In [2]:
!git clone https://github.com/BigRoddy/CMIN-Dataset.git

Cloning into 'CMIN-Dataset'...
remote: Enumerating objects: 350438, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 350438 (delta 0), reused 0 (delta 0), pack-reused 350411 (from 1)
Receiving objects: 100% (350438/350438), 404.05 MiB | 25.57 MiB/s, done.
Resolving deltas: 100% (41022/41022), done.
Updating files: 100% (369958/369958), done.


In [3]:
stock_path = "/content/CMIN-Dataset/CMIN-US/price/processed/AAPL.txt"
news_path ="/content/CMIN-Dataset/CMIN-US/news/preprocessed/AAPL"

In [4]:
import pandas as pd

In [5]:
#news_data = pd.read_csv(news_path)
stock_data = pd.read_csv(stock_path, sep='\t', header = None)

In [6]:
stock_data.iloc[:, 1] = stock_data.iloc[:, 1].astype(float)

In [7]:
def label_price(df):
    df['label'] = df.iloc[:, 1].apply(lambda x: 1 if x > 0 else 0)
    return df

In [8]:
stock_df = label_price(stock_data)

In [9]:
stock_df.rename(columns={0: 'date', 1: 'price'}, inplace=True)
# stock_df['date'] = pd.to_datetime(stock_df['date'])
stock_df

,date,price,2,3,4,5,6,label
0,2018-01-03,-0.000174,0.013928,0.013059,0.015952,-0.000174,118071600,0
1,2018-01-04,0.004645,0.000058,-0.006187,0.000698,0.004645,89738400,1
2,2018-01-05,0.011385,0.005216,0.010953,0.005637,0.011385,94640000,1
3,2018-01-08,-0.003714,0.005247,0.001369,0.005085,-0.003714,82271200,0
4,2018-01-09,-0.000115,0.001147,-0.003132,-0.002990,-0.000115,86336000,0
...,...,...,...,...,...,...,...,...
997,2021-12-17,-0.006502,-0.052153,-0.042343,-0.006208,-0.006502,195432700,0
998,2021-12-20,-0.008122,-0.009710,-0.016660,-0.013142,-0.008122,107499100,0
999,2021-12-21,0.019087,0.019491,0.015359,0.009913,0.019087,91185900,1
1000,2021-12-22,0.015319,0.008627,0.015358,0.017916,0.015319,92135300,1


In [10]:
min_value = stock_df.iloc[:, 0].min()
max_value = stock_df.iloc[:, 0].max()
print(f"Start date: {min_value}")
print(f"End date: {max_value}")

Start date: 2018-01-03
End date: 2021-12-23


In [11]:
new_df = "/content/CMIN-Dataset/CMIN-US/news/preprocessed/AAPL/2018-01-01"

In [12]:
import pandas as pd

# Replace 'file_path.jsonl' with your actual file path
data = pd.read_json(new_df, lines=True)

# Now 'data' is a DataFrame
print(data)

                                                text          created_at
0  [Forget, Your, iPhone, X,, Ignore, The, Samsun... 2018-01-01 23:28:00
1            [Apple's, Executive, Cash, Bonus, Plan] 2018-01-01 22:19:00
2  [2, Warren, Buffett, Stocks, to, Consider, Buy... 2018-01-01 21:08:20
3  [2, Warren, Buffett, Stocks, to, Consider, Buy... 2018-01-01 20:02:00
4  [Bitcoin, or, Stocks?, Here’s, the, One, to, B... 2018-01-01 16:06:40


In [13]:
!pip install torch transformers pandas scikit-learn

In [14]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [15]:
import pandas as pd
import glob
import os
import json

# Step 1: Set up file paths
json_directory = '/content/CMIN-Dataset/CMIN-US/news/preprocessed/AAPL/*'
# txt_file_path = 'path/to/stock_prices.txt'

# Step 2: Load JSON files
json_files = glob.glob(json_directory)
json_dataframes = []

for file in sorted(json_files):
    try:
        with open(file, 'r') as f:
            json_data = [json.loads(line) for line in f if line.strip()]  # Read each line
        df = pd.DataFrame(json_data)
        json_dataframes.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

combined_json_df = pd.concat(json_dataframes, ignore_index=True)
# combined_json_df = combined_json_df['created_at'].split(" ")[0]
combined_json_df['created_at'] = pd.to_datetime(combined_json_df['created_at'])

# Step 2: Extract the date
combined_json_df['created_at'] = combined_json_df['created_at'].dt.date
combined_json_df.rename(columns={'created_at': 'date'}, inplace=True)

# Combine text by date
# combined_json_df = combined_json_df.groupby('date')['text'].agg(lambda x: ' '.join(x)).reset_index()
combined_json_df = combined_json_df.groupby('date')['text'].agg(lambda x: ' '.join([' '.join(item) for item in x])).reset_index()

combined_json_df['date'] = pd.to_datetime(combined_json_df['date'])
combined_json_df


,date,text
0,2018-01-01,"Forget Your iPhone X, Ignore The Samsung And P..."
1,2018-01-02,Cramer reflects on how Trump's actions are fue...
2,2018-01-03,Apple Acquires Canadian Software Startup Wedne...
3,2018-01-04,Jimmy Iovine to leave Apple Music in August - ...
4,2018-01-05,CES kicks off with no lead women speakers or c...
...,...,...
1422,2021-12-27,Apple shutters 11 stores in New York City amid...
1423,2021-12-28,"Stock Market: As interest rates go up, options..."
1424,2021-12-29,The Dow's New High Is Hiding This Key Market R...
1425,2021-12-30,The Unsung Winners and Biggest Losers of China...


In [16]:
stock_df['date'] = pd.to_datetime(stock_df['date'])
stock_df

,date,price,2,3,4,5,6,label
0,2018-01-03,-0.000174,0.013928,0.013059,0.015952,-0.000174,118071600,0
1,2018-01-04,0.004645,0.000058,-0.006187,0.000698,0.004645,89738400,1
2,2018-01-05,0.011385,0.005216,0.010953,0.005637,0.011385,94640000,1
3,2018-01-08,-0.003714,0.005247,0.001369,0.005085,-0.003714,82271200,0
4,2018-01-09,-0.000115,0.001147,-0.003132,-0.002990,-0.000115,86336000,0
...,...,...,...,...,...,...,...,...
997,2021-12-17,-0.006502,-0.052153,-0.042343,-0.006208,-0.006502,195432700,0
998,2021-12-20,-0.008122,-0.009710,-0.016660,-0.013142,-0.008122,107499100,0
999,2021-12-21,0.019087,0.019491,0.015359,0.009913,0.019087,91185900,1
1000,2021-12-22,0.015319,0.008627,0.015358,0.017916,0.015319,92135300,1


In [17]:
# Step 3: Load the stock prices text file
# Adjust this depending on the structure of your txt file
stock_prices_df = stock_df

# Ensure the date column is in datetime format
#stock_prices_df['date'] = pd.to_datetime(stock_prices_df['date'])  # Adjust based on your column name

# Step 4: Merge DataFrames on date
final_df = pd.merge(combined_json_df, stock_prices_df, on='date', how='outer')  # or 'inner' based on your needs

# Optional: Save the final DataFrame to a file
# final_df.to_csv('combined_data.csv', index=False)

final_df.dropna(inplace=True)
final_df

,date,text,price,2,3,4,5,6,label
2,2018-01-03,Apple Acquires Canadian Software Startup Wedne...,-0.000174,0.013928,0.013059,0.015952,-0.000174,118071600.0,0.0
3,2018-01-04,Jimmy Iovine to leave Apple Music in August - ...,0.004645,0.000058,-0.006187,0.000698,0.004645,89738400.0,1.0
4,2018-01-05,CES kicks off with no lead women speakers or c...,0.011385,0.005216,0.010953,0.005637,0.011385,94640000.0,1.0
7,2018-01-08,Semiconductor Industry Meltdown Exposes Securi...,-0.003714,0.005247,0.001369,0.005085,-0.003714,82271200.0,0.0
8,2018-01-09,Canada's competition watchdog settles with Har...,-0.000115,0.001147,-0.003132,-0.002990,-0.000115,86336000.0,0.0
...,...,...,...,...,...,...,...,...,...
1428,2021-12-17,Apple (AAPL) Stock Moves -0.65%: What You Shou...,-0.006502,-0.052153,-0.042343,-0.006208,-0.006502,195432700.0,0.0
1431,2021-12-20,Amazon Slides Below Support Level as Big Tech ...,-0.008122,-0.009710,-0.016660,-0.013142,-0.008122,107499100.0,0.0
1432,2021-12-21,"Japan Pressed to Spend Billions, Cut Tax to Re...",0.019087,0.019491,0.015359,0.009913,0.019087,91185900.0,1.0
1433,2021-12-22,"After Austin blow, Cooper's office kept engagi...",0.015319,0.008627,0.015358,0.017916,0.015319,92135300.0,1.0


In [18]:
new_column_names = ['date', 'text','price', 'open', 'close','low', 'changes', 'volume', 'label']
final_df_renamed = final_df.set_axis(new_column_names, axis=1)

In [19]:
# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    final_df['text'].tolist(),
    final_df['label'].tolist(),
    test_size=0.2,
    random_state=42
)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [21]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.264800,0.250395
2,0.316300,0.243027
3,0.241000,0.224165
4,0.205700,0.238577
5,0.188600,0.254475


TrainOutput(global_step=495, training_loss=0.2432429648408986, metrics={'train_runtime': 107.7758, 'train_samples_per_second': 36.604, 'train_steps_per_second': 4.593, 'total_flos': 1037963793853440.0, 'train_loss': 0.2432429648408986, 'epoch': 5.0})

In [24]:
trainer.evaluate()

{'eval_loss': 0.25447481870651245,
 'eval_runtime': 1.5122,
 'eval_samples_per_second': 130.936,
 'eval_steps_per_second': 16.532,
 'epoch': 5.0}

In [25]:
def predict(text):
    # Check if CUDA is available and use the appropriate device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the model to the appropriate device
    model.to(device)

    # Tokenize the input text and move to the correct device
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    encoding = {key: val.to(device) for key, val in encoding.items()}  # Move to the same device as model

    with torch.no_grad():
        logits = model(**encoding).logits

    prediction = torch.argmax(logits, dim=-1).item()
    return prediction

# Example usage
sample_text = "Apple stock price will increase"
# Apple buys developer tool-maker Buddybuild Apple Inc. has acquired Buddybuild, a Vancouver, Canada-based company that makes software development tools, according to a Buddybuild blog. Apple stock is down a fraction to $172.05 in late trading. Buddybuild, Stocks Up Broadly, Apple Strong; Will This Bitcoin ETF Soar In 2018?U S stocks got off to a riveting start to the new year particularly in the tech sectors as Apple AAPL behaved the way one would expect for a leading stock that is testing support at a key technical juncture ibd display video id 3058957 width 50 float left autostart The FANG+ Stocks Are All At Key Technical Levels The Nasdaq surged 28 in 2017 as big name tech stocks continued to show their dominance How well did the FANG stocks perform and could there be more room for them to run in 2018 Here s the scorecard for Facebook FB Amazon AMZN Netflix 7 Tech Stocks Chase Coleman Is Buying Heading Into 2018 InvestorPlace Stock Market News Stock Advice amp Trading Tips At 42 years old Chase Coleman is possibly the youngest of the big league hedge fund managers In 2011 when he was 36 his Tiger Global long short equity fund returned over 45 That made it the most profitable hedge fund"
print(f"Sentiment: {predict(sample_text)}")


Sentiment: 0


# Stock prediction

In [26]:
!pip install tensorflow scikit-learn pandas

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error

# Load the stock dataset (adjust with your data)
df = final_df_renamed

# Features and target
X = df.drop(['text', 'price', 'date'], axis=1)
y = df['price']
X = X.astype(float)
y = y.astype(float)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a deep neural network model for stock prediction
nn_model = Sequential([
    Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
nn_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)
nn_model.save('nn_stock_model.h5')
# Evaluate the model
y_pred_nn = nn_model.predict(X_test_scaled)
mse_nn = mean_squared_error(y_test, y_pred_nn)
print(f"Neural Network MSE: {mse_nn}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 172ms/step - loss: 0.0071 - val_loss: 0.0019
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0011 - val_loss: 8.8436e-04
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.6376e-04 - val_loss: 5.7457e-04
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.5312e-04 - val_loss: 4.4362e-04
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6625e-04 - val_loss: 4.6214e-04
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5811e-04 - val_loss: 3.6805e-04
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2998e-04 - val_loss: 4.1542e-04
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1329e-04 - val_loss: 2.8080e-04
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0757e-04 - val_loss: 3.7142e-04
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0324e-04 - val_loss: 2.5660e-04
Epoch 11/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3657e-04 - val_loss: 4.3781e

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step
Neural Network MSE: 0.0005213950251017288


# Ensemble model

In [28]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import joblib

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)
# save the model
joblib.dump(rf_model, 'rf_stock_model.pkl')

# Get predictions from both models (Random Forest + Neural Network)
rf_preds = rf_model.predict(X_test_scaled)
nn_preds = nn_model.predict(X_test_scaled)

# Ensemble: Average the predictions
ensemble_preds = np.mean([rf_preds, nn_preds.flatten()], axis=0)

# Evaluate the ensemble model
mse_ensemble = mean_squared_error(y_test, ensemble_preds)
print(f"Ensemble Model MSE: {mse_ensemble}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Ensemble Model MSE: 0.00014061262302283368


# LSTM

In [29]:
!pip install tensorflow scikit-learn pandas

In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load stock price data (time-series data for forecasting)
df = final_df_renamed

# Prepare the time series data
df = df[['date', 'price']]  # Assuming 'Close' is the closing stock price
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['price'].values.reshape(-1, 1))

# Prepare the dataset for LSTM (timesteps)
def create_dataset(data, time_step=50):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

time_step = 50  # Using 50 days to predict the next
X, y = create_dataset(scaled_data, time_step)

# Reshape X for LSTM input (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split data into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the LSTM model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])

# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32)

# Save model
lstm_model.save('lstm_stock_model.h5')
# Predict and invert scaling for real prices
y_pred_lstm = lstm_model.predict(X_test)
y_pred_lstm_rescaled = scaler.inverse_transform(y_pred_lstm)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# Evaluate model performance
mse_lstm = mean_squared_error(y_test_rescaled, y_pred_lstm_rescaled)
print(f"LSTM Model MSE: {mse_lstm}")


<ipython-input-30-a7b51ab54efc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0800
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0084
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0081
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0081
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0081
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
LSTM Model MSE: 0.00019520323972415957


In [31]:
import pickle

# Assuming 'scaler' is your fitted scaler object
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Building Flask app

In [32]:
!pip install tensorflow scikit-learn pandas flask

In [33]:
from flask import Flask, request, jsonify
import numpy as np
import joblib
from tensorflow.keras.models import load_model
import networkx as nx

# Load the pre-trained models
rf_model = joblib.load('rf_stock_model.pkl')
nn_model = load_model('nn_stock_model.h5')
lstm_model = load_model('lstm_stock_model.h5')
scaler = joblib.load('scaler.pkl')

# Initialize Flask app
app = Flask(__name__)

# Define route for stock price prediction using ensemble model
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    features = np.array([data['feature1'], data['feature2'], data['feature3'],
                         data['feature4'], data['feature5'], data['feature6']]).reshape(1, -1)

    # Scale the features
    features_scaled = scaler.transform(features)

    # Get predictions from both models
    rf_pred = rf_model.predict(features_scaled)
    nn_pred = nn_model.predict(features_scaled)

    # Ensemble: Average the predictions
    final_pred = np.mean([rf_pred, nn_pred.flatten()])

    # Query the knowledge graph for recommendations based on the prediction
    recommendation = get_recommendation(final_pred)

    # Return prediction and recommendation as JSON
    return jsonify({
        'predicted_stock_price': final_pred,
        'recommendation': recommendation
    })

# Define route for stock price forecasting using LSTM model
@app.route('/forecast', methods=['POST'])
def forecast():
    data = request.json
    historical_data = np.array(data['historical_prices']).reshape(-1, 1)

    # Scale the historical data
    historical_data_scaled = scaler.transform(historical_data)

    # Reshape for LSTM input
    time_step = 50  # Same timestep used for LSTM training
    X = historical_data_scaled[-time_step:].reshape((1, time_step, 1))

    # Make a forecast for the next time step
    forecasted_price_scaled = lstm_model.predict(X)
    forecasted_price = scaler.inverse_transform(forecasted_price_scaled)

    # Return the forecasted price
    return jsonify({
        'forecasted_stock_price': forecasted_price[0][0]
    })

# Function to query the knowledge graph
def get_recommendation(prediction):
    G = nx.Graph()
    # Example knowledge graph for stock recommendations
    G.add_edge('Stock_Rising', 'Buy More', recommendation='Consider buying more stocks')
    G.add_edge('Stock_Falling', 'Hold', recommendation='Consider holding off on buying')
    G.add_edge('Stock_Falling', 'Sell', recommendation='Consider selling some stocks')

    # Simple recommendation logic
    if prediction > 100:
        return G['Stock_Rising']['Buy More']['recommendation']
    else:
        return G['Stock_Falling']['Sell']['recommendation']

# Run Flask app
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [38]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

# Create a DataFrame (assuming final_df_renamed is already defined)
news_df = final_df_renamed.head(3)

# Create a directed graph
G = nx.DiGraph()

# Build the graph from news headlines
for index, row in news_df.iterrows():
    words = row['text'].split()
    for i in range(len(words)):
        for j in range(i + 1, len(words)):
            if words[i] != words[j]:
                G.add_edge(words[i], words[j])